# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/video_games_sales.csv).

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:
- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('video_games_sales.csv')
df.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


### 1) Как критики относятся к спортивным играм?

Чтобы ответить на этот вопрос, нужно отобрать оценки критиков для спортивных игр и сравнить с какой-то пороговой оценкой. Будем считать, что если средние оценки будут превосходить 7.0, то игры критикам нравятся, если будут меньше - то уже не очень.

Сформулируем гипотезы:  
**Гипотеза Н0**: средняя оценка критиков = 7.0 (критики не очень любят спортивные игры)  
**Гипотеза Н1**: средняя оценка критиков > 7.0 (критикам нравятся спортивные игры)

Мы будем сравнивать значения выборки с константой, поэтому используем одновыборочный t-тест Стьюдента.

Уровень значимости возьмем стандартный, alpha = 0.05

In [3]:
critic_sport_score = df[(df['Genre'].str.contains('Sport'))&(~df.Critic_Score.isna())].Critic_Score
critic_sport_score

0        7.7
4        8.0
16       7.9
17       8.0
68       8.3
        ... 
50561    5.0
50730    9.2
50731    3.6
50840    4.0
51761    9.1
Name: Critic_Score, Length: 688, dtype: float64

In [4]:
from scipy import stats

In [5]:
alpha = 0.05
result = stats.ttest_1samp(critic_sport_score, 7.0)
print(result)

if (result.statistic > 0) & (result.pvalue / 2 < alpha):
    print('Отвергаем нулевую гипотезу,  средняя оценка критиков > 7.0')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка критиков <= 7.0 (критики не очень любят спортивные игры)')

Ttest_1sampResult(statistic=5.197156914436623, pvalue=2.671410902970997e-07)
Отвергаем нулевую гипотезу,  средняя оценка критиков > 7.0


Получили маленькое значение p-value, можем утверждать, что разница между оценками критиков и заданной нами константой статистически значима при нашем 0.05 уровне значимости. T-критерий больше 0, т.е. в среднем оценки критиков действительно больше заданной нами пороговой оценки.  
Вывод: в среднем критики хорошо относятся к спортивным играм

### 2) Критикам нравятся больше игры на PC или на PS4?

Сформулируем гипотезы:  
**Гипотеза Н0**: средние оценки критиков для PC и PS4 одинаковые  
**Гипотеза Н1**: средние оценки критиков для PC и PS4 различаются  

Сравниваем две выборки, используем двухвыборочный тест. Мы не знаем, одни и те же критики выставляли оценки для игр на разных платформах или нет. Но из общих соображений, предположу, что игры на PC оценивали одни люди, а на PS4 другие, т.е. выборки являются независимыми.

Уровень значимости alpha = 0.05

In [6]:
pc_scores = df[(df['Platform'].str.contains('PC'))&(~df.Critic_Score.isna())].Critic_Score
ps4_scores = df[(df['Platform'].str.contains('PS4'))&(~df.Critic_Score.isna())].Critic_Score

In [7]:
result = stats.ttest_ind(pc_scores, ps4_scores, equal_var=False)
print(result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному оценивают игры на PC и PS4')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, критики по-разному оценивают игры на PC и PS4


Вывод: критики по-разному относятся к играм на разных платформах. С учетом знака t-критерия (минус) средние оценки игр на PC меньше средних оценок игр на PS4, т.е. для критиков игры PS4 несколько более предпочтительны

### 3) Критикам больше нравятся стрелялки или стратегии?

Сформулируем гипотезы:  
**Гипотеза Н0**: средние оценки критиков для шутеров и стратегий одинаковые  
**Гипотеза Н1**: средние оценки критиков для шутеров и стратегий различаются  

Будем по-прежнему придерживаться идеи, что оценки критиков для одного и для другого жанра это независимые выборки. Используем двухвыборочный тест

Уровень значимости alpha = 0.05

In [8]:
shooter_scores = df[(df['Genre'].str.contains('Shooter'))&(~df.Critic_Score.isna())].Critic_Score
strategy_scores = df[(df['Genre'].str.contains('Strategy'))&(~df.Critic_Score.isna())].Critic_Score

In [9]:
result = stats.ttest_ind(shooter_scores, strategy_scores, equal_var=False)
print(result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному оценивают шутеры и стратегии')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу


По знаку t-критерия делаем вывод, что оценки у шутеров чуть хуже, чем у стратегий. Но значение p-value > alpha, поэтому мы не можем говорить, что эта разнича статистически значима.  
Вывод: критики примерно одинаково оценивают стратегии и шутеры

## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Привидите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:  

```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);      

7) Постройте модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), укажите `random_state=42`, оцените ее точность на тестовых данных;    

8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);    

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [10]:
spam = pd.read_csv('spam.csv')
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [12]:
spam.Category.unique()

array(['ham', 'spam'], dtype=object)

In [13]:
# 1. приведем все слова к нижнему регистру
spam['Message'] = spam['Message'].apply(lambda x: x.lower())
spam.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [14]:
# 2. уберем мусорные символы
spam['Message'] = spam.Message.str.replace('[\W_]+',' ', regex=True)
spam.head()

,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...


In [15]:
spam['Message'] = spam['Message'].apply(lambda x: x.split())
spam.head()

,Category,Message
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."


In [16]:
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

In [17]:
def drop_stopwords(row):
    return [word for word in row if word not in stopwords_set]
  

In [18]:
# 3. удалим из сообщений стоп-слова
spam['Message'] = spam['Message'].apply(drop_stopwords)

In [19]:
spam.head()

,Category,Message
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, goes, usf, lives, around, though]"


In [20]:
# 4. приведем слова в сообщениях к нормальной форме
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [21]:
def lemmatize_message(row):
    return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in row])

In [22]:
spam['Message'] = spam['Message'].apply(lemmatize_message)
spam.head()

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though


In [23]:
# 5. Преобразуем все сообщения в вектора TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(spam.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

tfidf_matrix

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
target = spam.Category

In [25]:
# 6. разделяем выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, target, test_size=0.3, random_state=42)

In [26]:
# 7. импортируем модуль и строим модель логической регрессии
from sklearn.linear_model import LogisticRegression

In [27]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [28]:
y_pred = lr.predict(X_test)
y_pred

array(['ham', 'ham', 'ham', ..., 'ham', 'spam', 'ham'], dtype=object)

In [29]:
y_test

3245     ham
944      ham
1044     ham
2484     ham
812      ham
        ... 
2505     ham
2525    spam
4975     ham
650     spam
4463     ham
Name: Category, Length: 1672, dtype: object

In [30]:
y_pred_series = pd.Series(y_pred)
y_pred_series.index = y_test.index

In [31]:
result = pd.concat([y_test, y_pred_series], axis=1)
result.columns = ['category_test', 'category_predicted']
result

,category_test,category_predicted
3245,ham,ham
944,ham,ham
1044,ham,ham
2484,ham,ham
812,ham,ham
...,...,...
2505,ham,ham
2525,spam,spam
4975,ham,ham
650,spam,spam


In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy_score(y_test, y_pred)

0.958732057416268

Получили довольно высокую оценку точности. Почти 96% сообщений из тестовых данных предсказаны верно

In [34]:
# 8) построим confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1445,    3],
       [  66,  158]], dtype=int64)

In [35]:
# фактические значения
y_test.value_counts()

ham     1448
spam     224
Name: Category, dtype: int64

In [36]:
# Предсказанные значения
y_pred_series.value_counts()

ham     1511
spam     161
dtype: int64

Из матрицы ошибок видим, что:
- 1445 сообщений типа ham были предсказаны верно (истинно-положительное решение), 
- 158 сообщений типа spam были предсказаны верно (истинно-отрицательное решение), 
- 66 сообщений содержат ошибку 1го рода (ложно-положительное решение). Исходное сообщение было типа spam, а модель его не распознала 
- 3 сообщения содержат ошибку 2го рода (ложно-отрицательное решение). Исходное сообщение было типа ham, а модель ошибочно отнесла его категорию к spam

In [37]:
# 9. Постройте датафрейм, который будет содержать исходные тексты сообщений, классифицированные неправильно 
# (с указанием фактического и предсказанного).

In [39]:
# Мы уже во многом изменили сообщения датафрефма spam, поэтому чтобы получить исходные тексты прочтем заново файл .csv  
spam_origin = pd.read_csv('spam.csv')
df_mistakes = result[result['category_test']!=result['category_predicted']]
df_mistakes = df_mistakes.join(spam_origin.drop('Category', axis=1), how='left')
df_mistakes

,category_test,category_predicted,Message
2952,ham,spam,Hey now am free you can call me.
881,spam,ham,Reminder: You have not downloaded the content ...
1961,spam,ham,Guess what! Somebody you know secretly fancies...
3864,spam,ham,Oh my god! I've found your number again! I'm s...
2575,spam,ham,Your next amazing xxx PICSFREE1 video will be ...
...,...,...,...
4543,spam,ham,FreeMsg Hi baby wow just got a new cam moby. W...
752,spam,ham,You have an important customer service announc...
309,spam,ham,TheMob> Check out our newest selection of cont...
495,ham,spam,Are you free now?can i call now?


In [41]:
df_mistakes_2 = result[result['category_test']!=result['category_predicted']]
df_mistakes_2 = df_mistakes_2.join(spam.drop('Category', axis=1), how='left')
df_mistakes_2

,category_test,category_predicted,Message
2952,ham,spam,hey free call
881,spam,ham,reminder downloaded content already paid goto ...
1961,spam,ham,guess somebody know secretly fancy wanna find ...
3864,spam,ham,oh god found number glad text back xafter msg ...
2575,spam,ham,next amazing xxx picsfree1 video sent enjoy on...
...,...,...,...
4543,spam,ham,freemsg hi baby wow got new cam moby wanna c h...
752,spam,ham,important customer service announcement premier
309,spam,ham,themob check newest selection content game ton...
495,ham,spam,free call


In [42]:
df_mistakes_2[(df_mistakes_2.category_test.str.contains('ham'))&(df_mistakes_2.category_predicted.str.contains('spam'))]

,category_test,category_predicted,Message
2952,ham,spam,hey free call
3692,ham,spam,sir waiting call free please call
495,ham,spam,free call


Интересно, что все сообщения, ошибочно принятые за спам, содержат ключевые слова "free call" - "бесплатный звонок", поэтому достаточно ожидаемо, что модель посчитала их спамом. Хотя до очистки от стоп-слов и лемматизации эти сообщения имели совершенно другой контекст и смысл.   

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
